In [7]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pandas import read_parquet
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pickle

In [8]:
df_street_busyness=pd.read_parquet('street_zones_factor.parquet')

In [9]:
unique_values=[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113, 114, 116, 120, 125, 127, 128, 137, 
               140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 
               231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]

In [10]:
# set an list for saving predict results
predictions = []
# input of the model, here just a sample
X_input = pd.DataFrame(np.array([[-11.2, 1, 2]]), columns=['temp', 'month', 'time_of_week'])

# loop all model 
for zone_id in unique_values:
    # load model
    filename = 'model-' + str(zone_id) + '.pkl'
    with open(filename, 'rb') as f:
        model = pickle.load(f)
    
    # predict
    prediction = model.predict(X_input)[0]
    # save reslut and zone ID into dictionary--prediction
    predictions.append({'Taxi_Zone_ID': zone_id, 'calm rate': prediction})

# transform dictionary into dataframe
df_predictions = pd.DataFrame(predictions)

In [19]:
def get_streets(df, street_zones):
    """
    Input: table with [taxi_zone] and [busyness_index]
        df = result of prediction
        street_zones = mapping table from 

    Output: table with street and adjusted busyness_index
    """


    # Perform the join.
    output_df =  pd.merge(df, street_zones, how='left',on='Taxi_Zone_ID')

    # Adjust busyness by the highway factor.
    output_df['street_calm_rate'] = 1 - (1 - output_df['calm rate']) * output_df['highway_factor']

    return output_df




new_result = get_streets(df_predictions, df_street_busyness)

In [21]:
new_result.head(10)

,Taxi_Zone_ID,calm rate,osmid,highway,geometry,highway_factor,street_calm_rate
0,4,0.401397,5668983,residential,"LINESTRING (-73.976212 40.725326, -73.97616 40...",0.6,0.640838
1,4,0.401397,194923763,tertiary,"LINESTRING (-73.976212 40.725326, -73.975266 4...",0.8,0.521118
2,4,0.401397,194923763,tertiary,"LINESTRING (-73.976212 40.725326, -73.977367 4...",0.8,0.521118
3,4,0.401397,5668983,residential,"LINESTRING (-73.975314 40.726545, -73.975738 4...",0.6,0.640838
4,4,0.401397,25167065,service,"LINESTRING (-73.975314 40.726545, -73.975402 4...",0.4,0.760559
5,4,0.401397,25167069,residential,"LINESTRING (-73.975314 40.726545, -73.975259 4...",0.6,0.640838
6,4,0.401397,438375431,secondary,"LINESTRING (-73.981345 40.72065, -73.981273 40...",1.0,0.401397
7,4,0.401397,25166891,residential,"LINESTRING (-73.979906 40.726866, -73.979828 4...",0.6,0.640838
8,4,0.401397,25166891,residential,"LINESTRING (-73.979906 40.726866, -73.979989 4...",0.6,0.640838
9,4,0.401397,813956617,tertiary,"LINESTRING (-73.979906 40.726866, -73.979867 4...",0.8,0.521118


In [ ]:
# save
new_result.to_parquet('predictions.parquet', index=False)